In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import netCDF4 as nc
import copy
import cartopy.crs as ccrs
import matplotlib.colors as colors
import sys

In [ ]:
# create a path temporarely to use ACOLITE AC
sys.path.append(r'\path\to\main\ACOLITE\directory')
import acolite as ac

# set input and output according to the ACOLITE documentation
inputfile = r'path\to\L1\image'
output_dir = r'path\to\output\directory'

# use all the settings for the ACOLITE AC according to the ACOLITE documentation
settings = {"inputfile":inputfile, "output":output_dir, "l2w_parameters":"rhow_*"}
ac.acolite.acolite_run(settings=settings)

In [ ]:
# load the results of the AC
a = r'path\to\AC\results'
os.chdir(a)

ds = nc.Dataset(AC_result.nc)

In [ ]:
# convert data to arrays
def convert_ds_variables_to_arrays(ds, field_name: str):
    return ds.variables[field_name][:]

red = convert_ds_variables_to_arrays(ds, 'red_band')
nir = convert_ds_variables_to_arrays(ds, 'nir_band')
lats = convert_ds_variables_to_arrays(ds, 'lat')
lons = convert_ds_variables_to_arrays(ds, 'lon')

In [ ]:
# set unrealistic outliers to nan values
def replace_outliers_by_nan(wavelength_array):
    wavelength_array_copy = wavelength_array.copy()
    unrealistic_wavelength = np.logical_or(wavelength_array_copy>5, wavelength_array_copy<0)
    wavelength_array_copy[unrealistic_wavelength] = np.nan
    return wavelength_array_copy

red = replace_outliers_by_nan(red)
nir = replace_outliers_by_nan(nir)

In [ ]:
# define the ANTA. use A and C parameters from Klein et al. (2021)
def calculate_ANTA(red, nir):
    output = np.zeros_like(red)
    mask = np.zeros_like(red)
    # get options
    T_low = red < 0.05
    T_mid = np.logical_and(red>=0.05, red<=0.07)
    T_nan = np.logical_or(np.isnan(red), np.isnan(nir))
    T_other = ~np.logical_or(T_low, T_mid, T_nan)

    # nan option
    r = red[T_nan]
    n = nir[T_nan]
    output[T_nan] = np.nan

    # intermediate option
    r = red[T_mid]
    n = nir[T_mid]
    output[T_mid] = ((1-(-50*r+3.5))*(A_red*r/(1-r/C_red)) + (-50*r+3.5)*A_nir*n/(1-n/C_nir))  

    # high option
    n = nir[T_other]
    output[T_other] = (A_nir*n/(1-n/C_nir))
    
     # low option
    r = red[T_low]
    output[T_low] = A_red * r / (1-r/C_red)
    
    # fill mask
    mask[T_other] = 3
    mask[T_nan] = 4
    mask[T_low] = 1
    mask[T_mid] = 2

    return output, mask

In [ ]:
output, mask = ANTA(red, nir)

In [ ]:
# convert outliers to nan
out_to_nan = np.logical_and(output > Z, output < 0) # defining Z as max. value is optinal and depends on your study area
output[out_to_nan] = np.nan

In [ ]:
# plot the results
fig = plt.figure(figsize=(20, 20), dpi=300)
fig.patch.set_alpha(1)
#fig.add_subplot(ROW,COLUMN,POSITION)
ax1 = fig.add_subplot(111, projection=ccrs.Mercator())
ax1.set_extent([X1, X2, Y1, Y2], crs=ccrs.PlateCarree()) # X1, X2, Y1, Y2 as coordinates
transform = ccrs.PlateCarree()

f1 = ax1.pcolormesh(lons, lats, np.ma.masked_invalid(output), 
                        shading='auto', transform = transform,
                        vmin=1, vmax=200, cmap=plt.cm.rainbow, norm=colors.LogNorm(vmin=1, vmax=200)) # log scale

fig.show()